In [1]:
import pandas as pd
import glob
import os
import os.path as op
from collections import Counter

from bisect import bisect_left
import numpy as np
from sklearn import neighbors
import pandas as pd
from math import log

from phage_count_table import phage_contig_table, map_clstr_raw
from nb_tools import readfa, swap_cluster_map

In [2]:
clstr = "/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90.fasta.clstr"
input_fa = "/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs.fasta"
phage_hits = "/mnt/scgc/simon/simonsproject/bats248_vs/phage_hits.csv"

In [3]:
#objects needed for generating the phage contig table per SAG in batch:

phage_hits_df = pd.read_csv(phage_hits)
cmap = swap_cluster_map(map_clstr_raw(clstr))

In [4]:
recruits = glob.glob("/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/*.csv")

In [5]:
to_start =  '''AG−891−A17
AG−892−P18
AG−893−J23
AG−894−C07
AG−895−P08
AG−897−A15  
AG−903−F19
AG−903−I06
AG−904−O13
AG−907−C19
AG−907−I10
AG−908−F15
AG-909-A05
AG−910−E05
AG−912−O18
AG−913−C05
AG−913−C17'''.replace("−","-").split()

recruits = [i for i in recruits if op.basename(i).split("_")[0] in to_start]

In [6]:
# SAG-specific files required:

ex_recruit = recruits[0]
ex_sag = op.basename(ex_recruit).split("_")[0]
ex_gff = "/mnt/scgc/simon/simonsproject/bats248_annotations/gff/{}.gff".format(ex_sag)
ex_faa = "/mnt/scgc/simon/simonsproject/bats248_annotations/faa/{}.faa".format(ex_sag)
assert op.exists(ex_gff)
assert op.exists(ex_faa)

In [7]:
rdf = pd.read_csv(ex_recruit).fillna(0)

# phage_contig_table(clstr_map, gff, phage_hits_df, outfile=None)
mdf = phage_contig_table(cmap, ex_gff, phage_hits_df).reset_index().fillna(0)

In [8]:
vsdf = pd.merge(mdf, rdf, on='contig', how='outer')

In [9]:
vsdf['ratio_virus_bacteria'] = vsdf['fr_mg-vir'] / vsdf['fr_mg-bac']

In [11]:
training_file = '/mnt/scgc_nfs/opt/viruscope/virus-training.csv'

In [85]:
k = 27
train_in = pd.read_table(training_file, sep = ',').dropna(axis = 0, how = 'any').reset_index()
train_label = list(train_in.iloc[:,-1].values)
train = train_in.loc[:,'viral_phage_gene_fraction':'ratio.virus.bacteria']
train.columns = ['viral_phage_gene_fraction','ratio_virus_bacteria']
# make the classifier and fit the trainign the data
clf = neighbors.KNeighborsClassifier(k, weights='distance')
clf.fit(train, train_label)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=27, p=2,
           weights='distance')

In [86]:
for_vclass = vsdf[['contig','viral_phage_gene_fraction','ratio_virus_bacteria']]
test = for_vclass.dropna()

In [87]:
# make the prediction (aka classify)
test['virus_class'] = clf.predict(test[['viral_phage_gene_fraction','ratio_virus_bacteria']])
probs = clf.predict_proba(test[['viral_phage_gene_fraction','ratio_virus_bacteria']])

/mnt/scgc_nfs/opt/common/anaconda3/4.0.0/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [88]:
test['prob 0'] = [i[0] for i in probs]
test['prob 1'] = [i[1] for i in probs]

/mnt/scgc_nfs/opt/common/anaconda3/4.0.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/mnt/scgc_nfs/opt/common/anaconda3/4.0.0/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [99]:
def pval_by_count(clf, test_set, training_df, 
                  test_cols=['viral_phage_gene_fraction','ratio_virus_bacteria'],
                 score_col='score'):
    ns = clf.kneighbors(test[test_cols], return_distance=False)
    pvals = []
    
    for l in ns:
        ndf = training_df.loc[l,:]
        pvals.append(Counter(ndf['score']).most_common()[0][1] / len(ndf))
    return pvals

In [100]:
pval_by_count(clf, test, train_in)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8888888888888888,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8518518518518519]

In [104]:
test

,contig,viral_phage_gene_fraction,ratio_virus_bacteria,virus_class,prob 0,prob 1
0,AG-891-A17_NODE_1,0.006024,0.135173,0.0,1.000000,0.000000
1,AG-891-A17_NODE_10,0.000000,0.181883,0.0,1.000000,0.000000
2,AG-891-A17_NODE_11,0.000000,0.118777,0.0,1.000000,0.000000
3,AG-891-A17_NODE_12,0.000000,0.076512,0.0,1.000000,0.000000
4,AG-891-A17_NODE_13,0.000000,0.160627,0.0,1.000000,0.000000
5,AG-891-A17_NODE_14,0.000000,0.125048,0.0,1.000000,0.000000
6,AG-891-A17_NODE_15,0.000000,0.093360,0.0,1.000000,0.000000
7,AG-891-A17_NODE_16,0.000000,0.094583,0.0,1.000000,0.000000
8,AG-891-A17_NODE_17,0.000000,0.362696,0.0,1.000000,0.000000
9,AG-891-A17_NODE_18,0.000000,0.051818,0.0,1.000000,0.000000


In [98]:
?clf.kneighbors

In [51]:
test[['viral_phage_gene_fraction','ratio_virus_bacteria']][31:32]

,viral_phage_gene_fraction,ratio_virus_bacteria
31,0.833333,373.609693


In [58]:
neighs[31]

array([2484, 2485, 2487, 2477, 2483, 2468,    9, 2488, 2476, 2472, 2463,
       2499, 2481, 2455, 2470, 2489, 2482, 2493, 2496, 2480, 2192, 2467,
       2471, 2464, 2501, 2420, 2492])

In [24]:
for i, l in test.iterrows():
    print(clf.kneighbors([l['viral_phage_gene_fraction'], l['ratio_virus_bacteria']]))
    if i == 3:
        break

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.0060241   0.13517296].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [123]:
test['prob_all'] = [l['prob 0'] if l['virus_class'] == 0 else l['prob 1'] for i, l in test.iterrows()]

/mnt/scgc_nfs/opt/common/anaconda3/4.0.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [224]:
def virus_class(vsdf, test_cols = ['viral_phage_gene_fraction','ratio_virus_bacteria'], 
                training_file = '/mnt/scgc_nfs/opt/viruscope/virus-training.csv', 
                k = 27, score_col='score'):
    ''' function for determining virus class using KNeighborsClassifier
    Args:
        vsdf (pandas.DataFrame): viruscope dataframe having the the two calculated test columns and a contigs column included
        test_cols (list): list of names for test columns to be used for KNeighbors Classifier
        training_file (path): path to the training file for the knn algorithm
        k (int): number of nearest neighbors
        score_col(str): name of score column to use for training
    Returns:
        dataframe with columns: contig, virus_class, probability
    '''
    def _pval_by_count(clf, test_set, training_df, 
                  test_cols=['viral_phage_gene_fraction','ratio_virus_bacteria'],
                 score_col='score'):
        
        ''' 
        function for calculating probability that matches the previously used R function 
        
        Args:
            clf: KNeighborsClassifier object
            test_set: dataframe used to test knn algorithm on
            training_df: dataframe used to train knn
            test_cols: columns in above dataframes used to train the knn algorithm
            score_col: column in training algorithm that reports the score
        Returns: 
            list of pvalues in same order as rows in test_set dataframe
        '''
        
        ns = clf.kneighbors(test[test_cols], return_distance=False)
        pvals = []

        for l in ns:
            ndf = training_df.loc[l,:]
            pvals.append(Counter(ndf['score']).most_common()[0][1] / len(ndf))
        return pvals
    
    train_in = pd.read_table(training_file, sep = ',').dropna(axis = 0, how = 'any').reset_index()
    train_in.rename(columns={'ratio.virus.bacteria':'ratio_virus_bacteria'}, inplace=True)
    train_in[test_cols[0]] = np.log10(train_in[test_cols[0]] + .001)
    train_in[test_cols[1]] = np.log10(train_in[test_cols[1]])
    train_label = list(train_in.iloc[:,-1].values)
    in_train = train_in[test_cols]
    in_train.columns = test_cols
    
    # make the classifier and fit the training the data
    
    clf = neighbors.KNeighborsClassifier(k, weights='distance', algorithm='auto')
    clf.fit(in_train, train_label)
    
    test = vsdf[['contig'] + test_cols].dropna()
    test[test_cols[0]] = np.log10(test[test_cols[0]] + .001)
    test[test_cols[1]] = np.log10(test[test_cols[1]])
    
    test['virus_class'] = clf.predict(test[test_cols])
    probs = clf.predict_proba(test[test_cols])
    # built in probability function
    #test['prob 0'] = [i[0] for i in probs]
    #test['prob 1'] = [i[1] for i in probs]
    #test['prob_all'] = [l['prob 0'] if l['virus_class'] == 0 else l['prob 1'] for i, l in test.iterrows()]
    
    test['virus_prob'] = _pval_by_count(clf, test, train_in, test_cols, score_col)
    
    return test[['contig','virus_class','virus_prob']]

In [225]:
test_df = pd.read_csv("/mnt/scgc/simon/simonsproject/jb_vs_test/AG-891/AG-891-A17/summary/AG-891-A17_contigs-summary.csv.gz")

test_df.rename(columns={'Unnamed: 0':'contig'}, inplace=True)

In [226]:
test_out = virus_class(test_df, test_cols = ['viral_phage_gene_fraction','ratio_virus_bacteria'])
test_out

,contig,virus_class,virus_prob
0,AG-891-A17_NODE_1,0.0,1.000000
1,AG-891-A17_NODE_2,0.0,1.000000
2,AG-891-A17_NODE_3,0.0,1.000000
3,AG-891-A17_NODE_4,0.0,1.000000
4,AG-891-A17_NODE_5,0.0,1.000000
5,AG-891-A17_NODE_6,0.0,1.000000
6,AG-891-A17_NODE_7,0.0,1.000000
7,AG-891-A17_NODE_8,0.0,1.000000
8,AG-891-A17_NODE_9,1.0,0.962963
9,AG-891-A17_NODE_10,0.0,1.000000


In [174]:
%load_ext rpy2.ipython

In [175]:
%R library(dplyr); library(class)

/mnt/scgc_nfs/opt/common/anaconda3/4.0.0/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/mnt/scgc_nfs/opt/common/anaconda3/4.0.0/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/mnt/scgc_nfs/opt/common/anaconda3/4.0.0/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)


array(['class', 'dplyr', 'tools', 'stats', 'graphics', 'grDevices',
       'utils', 'datasets', 'methods', 'base'], 
      dtype='<U9')

In [182]:
%%R -i test_df -i train_in
trainin <- train_in[,c('viral_phage_gene_fraction','ratio.virus.bacteria')]
scores <- train_in[,c('score')]

In [186]:
test_df.columns

Index(['contig', 'contig_length', 'gene_count', 'viral_phage_gene_count',
       'viral_phage_gene_fraction', 'viral2_phage_gene_count',
       'viral2_phage_gene_fraction', 'Similarity_1.LineP.all.fr',
       'Similarity_1.POV.fr', 'ratio_virus_bacteria', 'virus_class',
       'virus_prob'],
      dtype='object')

In [198]:
%%R
to_test <- na.omit(test_df[,c('contig','viral_phage_gene_fraction','ratio_virus_bacteria')])

In [200]:
%%R
knn(trainin, to_test[,c('viral_phage_gene_fraction','ratio_virus_bacteria')], scores, k=27, prob=TRUE)

 [1] 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
attr(,"prob")
 [1] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
 [8] 1.0000000 0.8518519 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
[15] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
[22] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
[29] 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000 1.0000000
[36] 1.0000000 1.0000000 0.8888889
Levels: 0 1


Great.  These values are the same as the neighbor rank approach!  weird that it differs from the values calculated from the viruscope output... something to look into.

NOTE from the future: ... talked to Ben about this and looked back at R source, found that he and Nick input the log10 of the training values into the knn algorithm.  